In [7]:
import pandas as pd 
import os 
yes=[]
no=[]

for filename in os.listdir("./clean_database"):
        did_it=False
        filepath = os.path.join("./clean_database", filename)
        dataframe=pd.read_csv(filepath,sep="\t") 
        
        price_at_one_hour_left = dataframe.iloc[29]["token_price_usd"] 
        
        for index in range(30, len(dataframe)):
            price_after_one_hour = dataframe.iloc[index]["token_price_usd"]
            if((price_at_one_hour_left/price_after_one_hour)>=2):
                  
                  yes.append(dataframe.iloc[index]["token_address"])
                  did_it=True
                  break
        if(did_it==False):
              no.append(dataframe.iloc[29]["token_address"])   

print("number of token did a x3",len(yes))
print("number of token didn't a x3",len(no))

number of token did a x3 126
number of token didn't a x3 172


In [8]:
X=[]
Y=[]
for filename in os.listdir("./clean_database"):
        did_it=False
        filepath = os.path.join("./clean_database", filename)
        dataframe=pd.read_csv(filepath,sep="\t") 
        sample= []
        token_adress= dataframe.iloc[29]["token_address"]

        for index in range(0, 30):
                sample.append([dataframe.iloc[index]["token_m5_buys"]/300,dataframe.iloc[index]["token_m5_sells"]/300,dataframe.iloc[index]["token_h1_buys"]/500,dataframe.iloc[index]["token_h1_sells"]/500])
        X.append(sample)
        if(token_adress in yes):
                Y.append(1)
        else : 
                Y.append(0)  

In [13]:
from sklearn.model_selection import train_test_split

# Assuming X_seqs is your sequence data and y_labels is the corresponding labels
# X_seqs is a list of sequences
# y_labels is a list of corresponding labels (1 for positive, 0 for negative)

# Split the data into training and test sets with stratified sampling
X_train_seqs, X_test_seqs, y_train_labels, y_test_labels = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# Print the class distribution in the training and test sets
print("Training set class distribution:")
print("Label 1:", sum(y_train_labels), ", Label 0:", len(y_train_labels) - sum(y_train_labels))
print("Test set class distribution:")
print("Label 1:", sum(y_test_labels), ", Label 0:", len(y_test_labels) - sum(y_test_labels))


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# Convert X to a numpy array
X_arr = np.array(X_train_seqs)

# Convert y to a numpy array
y_arr = np.array(y_train_labels)

# Define model architecture for binary classification
model = Sequential()
model.add(LSTM(units=150, input_shape=(30,len(X[0][0])), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))  # Use sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])  # Use binary_crossentropy loss for binary classification

# Train the model
history = model.fit(X_arr, y_arr, epochs=20, batch_size=32)

Epoch 1/20
8/8 [==============================] - 2s 20ms/step - loss: 0.7108 - accuracy: 0.5252
Epoch 2/20
8/8 [==============================] - 0s 20ms/step - loss: 0.6890 - accuracy: 0.5840
Epoch 3/20
8/8 [==============================] - 0s 20ms/step - loss: 0.6708 - accuracy: 0.6008
Epoch 4/20
8/8 [==============================] - 0s 20ms/step - loss: 0.6646 - accuracy: 0.6050
Epoch 5/20
8/8 [==============================] - 0s 19ms/step - loss: 0.6587 - accuracy: 0.5966
Epoch 6/20
8/8 [==============================] - 0s 19ms/step - loss: 0.6514 - accuracy: 0.5924
Epoch 7/20
8/8 [==============================] - 0s 20ms/step - loss: 0.6420 - accuracy: 0.6218
Epoch 8/20
8/8 [==============================] - 0s 21ms/step - loss: 0.6500 - accuracy: 0.5840
Epoch 9/20
8/8 [==============================] - 0s 22ms/step - loss: 0.6433 - accuracy: 0.6471
Epoch 10/20
8/8 [==============================] - 0s 20ms/step - loss: 0.6364 - accuracy: 0.6008
Epoch 11/20
8/8 [============

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

X_arr = np.array(X_test_seqs)
predictions = model.predict(X_arr)
binary_predictions = (predictions > 0.5).astype(int)
# If your model outputs probabilities (e.g., if using sigmoid activation in the last layer), 
# you might need to threshold them to get binary predictions
y_arr=y_test_labels
        
predictions=binary_predictions

# Calculate metrics
accuracy = accuracy_score(y_arr, predictions)
precision = precision_score(y_arr, predictions)
recall = recall_score(y_arr, predictions)
f1 = f1_score(y_arr, predictions)

# Print metrics

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Assuming you have true labels (y_test) and predictions (binary_predictions) ready
report = classification_report(y_arr, predictions, target_names=['Class 0', 'Class 1'])

print("\n")
print(report)

2/2 [==============================] - 1s 9ms/step
10
Accuracy: 0.65
Precision: 0.7
Recall: 0.28
F1 Score: 0.4


              precision    recall  f1-score   support

     Class 0       0.64      0.91      0.75        35
     Class 1       0.70      0.28      0.40        25

    accuracy                           0.65        60
   macro avg       0.67      0.60      0.58        60
weighted avg       0.67      0.65      0.61        60

